In [ ]:
library(tidyverse)
library(lubridate)
library(tidymodels)
library(ggplot2)
library(readxl)

In [ ]:
players <- read_excel("players.xlsx")

sessions <- read_csv("sessions.csv")

In [ ]:
sessions_clean <- sessions |>
mutate(start_datetime = dmy_hm(start_time), end_datetime = dmy_hm(end_time), 
       hour_of_day = hour(start_datetime), 
       day_of_week = wday(start_datetime, label = TRUE, week_start = 1),
       date = date(start_datetime)) |>
filter(!is.na(start_datetime), !is.na(end_datetime))
sessions_clean
#data cleaning and separation

In [ ]:
hourly_activity <- sessions_clean |>
count(hour_of_day) |>
arrange(desc(n))
print(hourly_activity) 

In [ ]:
daily_activity <- sessions_clean |>
count(day_of_week) |>
arrange(desc(n))
print(daily_activity)

In [ ]:
time_points <- seq(from = floor_date(min(sessions_clean$start_datetime), "hour"),
                to = ceiling_date(max(sessions_clean$end_datetime), "hour"), by = "1 hour")

In [ ]:
simultaneous_analysis <- tibble(datetime = time_points) |>
mutate(player_count = map_dbl(datetime, ~ sum(sessions_clean$start_datetime <= .x & sessions_clean$end_datetime >= .x)),
       hour = hour(datetime), day = wday(datetime, label = TRUE, week_start = 1))
print(simultaneous_analysis)

In [ ]:
peak_simultaneous_hours <- simultaneous_analysis |>
group_by(hour) |>
summarize(avg_players = mean(player_count), max_players = max(player_count),
          peak_probability = mean(player_count > quantile(player_count, 0.75))) |>
arrange(desc(avg_players))
print(peak_simultaneous_hours)

In [ ]:
hour_day_activity <- sessions_clean |>
count(day_of_week, hour_of_day) |>
arrange(desc(n))
print(hour_day_activity)

In [ ]:
hourly_plot <- hourly_activity |>
ggplot(aes(x = factor(hour_of_day), y = n)) +
geom_col(fill = "blue", alpha = 0.7) +
labs(title = "Gaming sessions by hour of day", x = "Hour of day in 24 hour format",
     y = "No. of sessions") +
theme(element_text(size = 20)) 

hourly_plot

In [ ]:
daily_plot <- daily_activity |>
ggplot(aes(x = factor(day_of_week), y = n)) +
geom_col(fill = "red", alpha = 0.7) +
labs(title = "Gaming sessions by Day of Week", x = "Day of Week",
     y = "No. of sessions") +
theme(element_text(size = 20))

daily_plot

In [ ]:
hour_day_heatmap <- sessions_clean |>
count(day_of_week, hour_of_day)

heatmap_plot <- hour_day_heatmap |>
ggplot(aes(x = factor(hour_of_day), y = day_of_week, fill = n)) +
geom_tile() + scale_fill_gradient(low = "white", high = "red") +
labs(title = "Activity: Hour vs Day of Week", x = "Hour of Day", y = "Day of Week", fill = "Sessions") +
theme(element_text(size = 20))
heatmap_plot

In [ ]:
peak_simultaneous_plot <- peak_simultaneous_hours |>
ggplot(aes(x = factor(hour), y = avg_players)) +
geom_col(fill = "green", alpha = 0.7) +
labs(title = "Average Simultaneous Players by Hour", x = "Hour of Day",
     y = "Average Simultaneous Players") +
theme(element_text(size = 20))

peak_simultaneous_plot

In [ ]:
top_periods <- hour_day_activity %>% head(10)
top_period_plot <- top_periods |>
ggplot(aes(x = reorder(paste(day_of_week, hour_of_day, sep = " "), n), y = n)) +
geom_col(fill = "orange", alpha = 0.7) + 
labs(title = "Top 10 time periods", x = "Day and Hour", y = "No. of Sessions") + 
theme(element_text(size = 20))

top_period_plot

In [ ]:
simultaneous_plot <- simultaneous_analysis |>
ggplot(aes(x = datetime, y = player_count)) +
geom_line(color = "red", alpha = 0.7) +
geom_smooth(method = "loess", color = "blue", se = FALSE) +
labs(title = "Simultaneous Players over Time", x = "Date and Time", y = "No. of Simultaneous Players") +
theme(element_text(size = 20))

simultaneous_plot

In [ ]:
critical_periods <- hour_day_activity |>
filter(n > quantile(hour_day_activity$n, 0.8))
print(critical_periods)

In [ ]:
primary_peak_hours <- peak_simultaneous_hours |>
filter(avg_players > mean(peak_simultaneous_hours$avg_players))
print(primary_peak_hours)

Written Proposal (500 words maximum):

This assignment focuses on the broad question of Demand Forecasting (Question3): Predicting the time windows that are most likely to have a large number of simultaneous players in order to optimize license provisioning. The dataset comes with 2 files: "sessions.csv" and "players.xlsx". The sessions file contains individual player game sessions from start_time to end_time with the hashed emails as player identifiers. Variables like hour_of_day and day_of_week were extracted as important timetamps. The players file contains unique player records with their descriptive features like player age and gender. Since the core analysis is time-based the sessions file is used to derive time-series activity. A potential issue is an inherent lack of precision in the start_time and end_time not accurate enough to reflect the true time a player joins/leaves, it introduces uncertainty into the concurrent player count thus making the server demand data inaccurate and can be addressed through aggregating the concurrent count into large time bins to smooth out minor timing errors.

The specific question formulated is "What specific 1 hour window on which days of the week are most likely to experience a server load above the 75th percentile of average concurrent player count?", the question identifies the concurrent player count (response variable) and the explanatory variables: hour_of_day and day_of_week. The relationship between time and load is visualized through plots. The overall activity is highest on Saturdays and Sundays and during the 2 time windows of 12am - 4am and 6pm - 11pm. The hours with highest concurrent players are in the late night and early morning (1am, 3am and 5am), peaking at 3am with ~0.82 players. This pattern validates the time-based variables usage for modelling. 

The method to predict the relationship is using a Multiple Linear Regression (MLR) model